In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.set_printoptions(precision=3, linewidth=500, threshold=500, suppress=True, edgeitems=5)

%load_ext autoreload
%autoreload 2
# %matplotlib notebook

In [2]:
from ndgsp.graph.graphs import Graph, ProductGraph

2023-02-03 15:24:15.642885: E external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


Using Jax backend with device TFRT_CPU_0


In [3]:
from numpy import eye as I
J = lambda N: np.ones((N, N))

In [4]:
np.random.seed(0)

In [5]:
def check_rule(A_A: np.ndarray, 
               A_B: np.ndarray, 
               a: int, 
               b: int, 
               a_: int, 
               b_: int, 
               rule: int):
    """
    For a given product graph rule, check whether two nodes (a, b) and (a_, b_)
    are connected in the product of graphs with adjacency matrices A_A and A_B
    """
    
    if rule == 1:
        return A_A[a, a_] and b == b_
    
    if rule == 2:
        return not A_A[a, a_] and b == b_
    
    if rule == 3:
        return A_A[a, a_] and A_B[b, b_]
    
    if rule == 4:
        return not A_A[a, a_] and A_B[b, b_]
    
    if rule == 5:
        return A_A[a, a_] and not A_B[b, b_]
    
    if rule == 6:
        return not A_A[a, a_] and not A_B[b, b_]
    
    if rule == 7:
        return a == a_ and A_B[b, b_]
    
    if rule == 8:
        return a == a_ and not A_B[b, b_]

    raise ValueError('rule should be an int from 1-8')


def get_A_from_rules(A_A: np.ndarray, 
                     A_B: np.ndarray, 
                     rules: list[int]):
    """
    Create a product graph adjacency matrix using the factor 
    adjacency matrices and a set of rules
    """
    
    A = len(A_A)
    B = len(A_B)
    
    out = np.zeros((A * B, A * B))

    nodes = np.argwhere(np.ones((A, B)))
    
    for i, (a, b) in enumerate(nodes): 
        
        for j, (a_, b_) in enumerate(nodes):
            
            out[i, j] = out[j, i] = any(check_rule(A_A, A_B, a, b, a_, b_, rule) for rule in rules)
            
    return out


def get_A_from_adjacency(A_A: np.ndarray,
                         A_B: np.ndarray,
                         product: str):
    """
    Create a product graph adjacency matrix for a given product
    using the factor adjacency matrices
    """

        
    NA = len(A_A)
    NB = len(A_B)
    
    if product == 'Cartesian':
        
        return np.kron(A_A, I(NB)) + np.kron(I(NA), A_B)
    
    if product == 'direct':
        
        return np.kron(A_A, A_B)
    
    if product == 'strong':
        
        return np.kron(A_A, I(NB)) + np.kron(I(NA), A_B) + np.kron(A_A, A_B)
    
    if product == 'lexicographic':
        
        return np.kron(A_A, J(NB)) + np.kron(I(NA), A_B)


def check_A_consistent():
    """
    Check that constructing the product graph adjacency from the rules 
    and from the adjacency matrix products result in the same graph
    """

    GA = Graph.random_connected(8)
    GB = Graph.random_connected(10)

    for product, rules in [('Cartesian', [1, 7]), 
                        ('direct', [3]), 
                        ('strong', [1, 3, 7]), 
                        ('lexicographic', [1, 3, 5, 7])]:

        A1 = get_A_from_rules(GA.A, GB.A, rules)
        A2 = get_A_from_adjacency(GA.A, GB.A, product)

        assert(np.allclose(A1, A2))

    return True




In [6]:
check_A_consistent()

True

In [7]:
    
def get_L(A_A: np.ndarray, 
          A_B: np.ndarray, 
          L_A: np.ndarray, 
          L_B: np.ndarray, 
          product: str):
    """
    Get the product graph Laplacian matrix
    """

    NA = len(A_A)
    NB = len(A_B)
    
    D_A = np.diag(A_A.sum(0))
    D_B = np.diag(A_B.sum(0))
           
    if product == 'Cartesian':
        
        return np.kron(L_A, I(NB)) + np.kron(I(NA), L_B)
    
    if product == 'direct':
        
        return np.kron(D_A, L_B) + np.kron(L_A, D_B) - np.kron(L_A, L_B)
    
    if product == 'strong':
        
        return np.kron(L_A, I(NB)) + np.kron(I(NA), L_B) + np.kron(D_A, L_B) + np.kron(L_A, D_B) - np.kron(L_A, L_B)
    
    if product == 'lexicographic':
        
        return np.kron(I(NA), L_B) + np.kron(L_A, J(NB)) + np.kron(D_A, NB * I(NB) - J(NB))  
    

def check_L_consistent():
    """
    Check that the kronecker construction methods for the product graph 
    Laplacian are consistent
    """

    GA = Graph.random_connected(8)
    GB = Graph.random_connected(10)

    for product in ['Cartesian', 'direct', 'strong', 'lexicographic']:

        L1 = get_L(GA.A, GB.A, GA.L, GB.L, product)
        
        A2 = get_A_from_adjacency(GA.A, GB.A, product)
        L2 = np.diag(A2.sum(0)) - A2

        assert(np.allclose(L1, L2))

    return True


In [8]:
check_L_consistent()

True

In [12]:
def get_product_eigs(L_A: np.ndarray, L_B: np.ndarray, product: str):
    """
    Given the factor Laplacians, return the eigenvalues and eigenvectors of 
    the product Laplacian for a given product type. 

    Note that the direct and strong solutions only work for regular factor graphs
    """

    lamA, UA = np.linalg.eigh(L_A)
    lamB, UB = np.linalg.eigh(L_B)

    if product == 'Cartesian':

        lam = np.kron(lamA, np.ones_like(lamB)) + np.kron(np.ones_like(lamA), lamB)
        U = np.kron(UA, UB)

    if product == 'direct':

        rA = -L_A[0, 1:].sum()
        rB = -L_B[0, 1:].sum()
        lam = rB * np.kron(lamA, np.ones_like(lamB)) + rA * np.kron(np.ones_like(lamA), lamB) - np.kron(lamA, lamB)
        U = np.kron(UA, UB)

    if product == 'strong':
        
        rA = -L_A[0, 1:].sum()
        rB = -L_B[0, 1:].sum()
        lam = (1 + rB) * np.kron(lamA, np.ones_like(lamB)) + (1 + rA) * np.kron(np.ones_like(lamA), lamB) - np.kron(lamA, lamB)
        U = np.kron(UA, UB)

    if product == 'lexicographic':

        dA = -np.tril(L_A, -1).sum(0) - np.triu(L_A, 1).sum(0)
        lam1 = len(L_B) * np.kron(dA, np.ones_like(lamB[1:])) + np.kron(np.ones_like(lamA), lamB[1:])
        U1 = np.kron(I(len(L_A)), UB[:, 1:])

        lam2 = lamA * len(L_B)
        U2 = np.array([np.kron(UA[:, i], np.ones(len(L_B))) for i in range(len(lamA))]).T
        U2 /= np.linalg.norm(U2, axis=0)

        lam = np.concatenate([lam1, lam2])
        U = np.concatenate([U1, U2], axis=1)

    return lam, U


In [14]:
def check_eigs_consistent(): 

    for product in ['Cartesian', 'direct', 'strong', 'lexicographic']:

        if product in ['strong', 'direct']:

            GA = Graph.random_regular(6, 3)
            GB = Graph.random_regular(8, 3)

        else:
            GA = Graph.random_connected(6)
            GB = Graph.random_connected(8)


        lam, U = get_product_eigs(GA.L, GB.L, product)
        L = get_L(GA.A, GB.A, GA.L, GB.L, product)

        assert np.allclose(U @ np.diag(lam) @ U.T, L)

    return True


check_eigs_consistent()


True